In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [58]:
torch.manual_seed(1)

## Softmax Regression / Multi-Class classification

### 다중 클래스 분류(Multi-class Classification)

`로지스틱 회귀`는 2개의 선택지 중에서 1개를 고르는 `이진 분류(Binary Classification)`

`소프트맥스 회귀`는 3개 이상의 선택지 중에서 1개를 고르는 `다중 클래스 분류(Multi-Class Classification)`

Iris 데이터셋을 이용한 문제는 `꽃받침길이(SepalLength)`, `꽃받침넓이(SepalWidth)`, `꽃잎길이(PetalLength)`, `꽃잎넓이(PetalWidth)`라는 4개의 `특성(feature)`로부터 

`setosa`, `versicolor`, `virginica` 라는 3개의 붓꽃 품종 중 어떤 `품종(Species)`인지를 예측하는 문제

### 소프트맥스 회귀(Softmax Regression)

소프트맥스 회귀는 각 클래스마다 소수 확률을 할당, 이때 총 확률의 합은 1이고 각 클래스가 정답일 확률이 됨, 해당 과정을 다중 클래스 분류 문제에 적용

소프트맥스 회귀는 선택지의 개수만큼의 차원을 가지는 벡터를 만들고, 해당 벡터가 벡터의 모든 원소의 합이 1이 되도록 원소들의 값을 변환시켜야 함

이 변환 함수를 `소프트맥스 함수(Softmax function)`라 함

$ H(X) = softmax(WX + B) $

### 소프트맥스 함수(Softmax function)

소프트맥스 함수는 분류해야하는 클래스의 총 개수를 k라고 할 때, k차원의 벡터를 입력받아 각 클래스에 대한 확률을 추정

k차원의 벡터에서 i번째 원소는 $z_i$, i번째 클래스가 정답일 확률을 $p_i$로 나타낼때 소프트맥스 함수 $p_i$는 다음과 같음

$ p_i = \frac{e^{z_i}}{\sum_{j=1}^{k}e^{z_j}} for \, i = 1,2,...,k $

위의 붓꽃 문제의 경우 k=3 이므로 3차원 벡터 $z=[z_1 z_2 z_3]$의 입력을 받는 경우 아래와 같이 사용된다

$ softmax(z) = [ \frac{e^{z_1}}{\sum_{j=1}^{3}e^{z_j}} \frac{e^{z_2}}{\sum_{j=1}^{3}e^{z_j}} \frac{e^{z_3}}{\sum_{j=1}^{3}e^{z_j}} ] = [p_1, p_2, p_3] = \hat{y} $

p각각은 각 클래스가 정답일 확률을 나타내며 0과 1사이의 값으로 총 합이 1이 된다.

샘플 데이터는 4개의 독립 변수 x를 가지는데 이는 모델이 4차원 벡터를 입력으로 받음을 의미함

소프트맥스의 함수의 입력으로 사용되는 벡터는 벡터의 차원이 분류하고자 하는 클래스의 개수가 되어야 함

샘플 데이터 벡터를 소프트맥스 함수의 입력 벡터로 차원을 축소하는 방법으로 

소프트맥스 함수의 입력 벡터 z의 차원수만큼 결과값의 나오도록 가중치 곱을 진행, 총(4 x 3 = 12)개의 다른 가중치를 가지고 학습 과정에서 점차적으로 오차를 최소화하는 가중치로 값이 변경

예측값과 비교를 할 수 있는 실제값의 표현 방법으로 원-핫 벡터를 사용, 예측값을 원-핫 인코딩을 통해 원-핫 벡터로 수치화한 후 오차를 계산함

softmax( W(c * f) * X(f * 1) + B(c * 1) ) = Y(c * 1)

이때 f는 특성의 수, c는 클래스의 개수에 해당

### 비용 함수(Cost function)

소프트맥스 회귀에서는 비용 함수로 `크로스 엔트로피(cross entropy)` 함수를 사용

실제값이 y, 원-핫 벡터로 표현된 실제값의 j번째 인덱스를 y_j, 클래스의 개수가 k, j번째 클래스일 확률이 p_j일때 수식으로 나타내면 아래와 같다

$ cost(W) = - \displaystyle\sum_{j=1}^{k} y_j log(p_j) $

c가 실제값 원-핫 벡터에서 1을 가진 원소의 인덱스라 하면, $p_c = 1$은 $\hat{y}$가 $y$를 정확하게 예측한 경우이다

식에 대입해보면 $ -1log(1) = 0 $으로 결과적으로 크로스 엔트로피 함수의 값은 0이 된다

따라서 해당 값을 최소화 하는 방향으로 학습해야 한다

n개 전체 데이터에 대한 평균을 구할때 최종 비용함수는

$ cost(W) = - \frac{1}{n} \displaystyle\sum_{i=1}^{n} \displaystyle\sum_{j=1}^{k} y_j^{(i)} log(p_j^{(i)}) $

### PyTorch로 비용함수 구현

In [59]:
z = torch.FloatTensor([1, 2, 3])
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
print(hypothesis.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [60]:
# low-level 구현
# 임의의 3 * 5 행렬 크기 텐서 생성
z = torch.rand(3, 5, requires_grad=True)

# 두번째 차원에 대해서 소프트맥스 함수를 적용한다는 의미에서 dim=1
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

# 각 샘플에 대해서 임의의 레이블
y = torch.randint(5, (3,)).long()
print(y)

# 각 레이블에 대해서 원-핫 인코딩을 수행
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)
print(y_one_hot)

# 비용 함수 식에 따라 구현
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)
tensor([0, 2, 1])
tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
tensor(1.4689, grad_fn=<MeanBackward0>)


In [61]:
# high-level 구현
# F.softmax() + torch.log() = F.log_softmax()
# torch.log(F.softmax(z, dim=1)) -> F.log_softmax(z, dim=1)
print(torch.log(F.softmax(z, dim=1)))
print(F.log_softmax(z, dim=1))

# F.log_softmax() + F.nll_loss() = F.cross_entropy()
# (y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean() 
# -> (y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()
# -> F.nll_loss(F.log_softmax(z, dim=1), y)
# -> F.cross_entropy(z, y)
print((y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean())
print((y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean())
print(F.nll_loss(F.log_softmax(z, dim=1), y))
print(F.cross_entropy(z, y)) # F.cross_entropy는 비용 함수에 소프트맥스 함수까지 포함

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)
tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)
tensor(1.4689, grad_fn=<MeanBackward0>)
tensor(1.4689, grad_fn=<MeanBackward0>)
tensor(1.4689, grad_fn=<NllLossBackward0>)
tensor(1.4689, grad_fn=<NllLossBackward0>)


### 이진 분류에서의 크로스 엔트로피 함수

로지스틱 회귀에서 사용한 이진 크로스 엔트로피(BCE) 함수식과 결과적으로는 동일하다

$ cost(H(x), y) = -[y log H(x) + (1-y) log (1-H(x))] $

위 식에서 $y$를 $y_1$, $1-y$를 $y_2$를 치환하고 $H(x)$를 $p_1$, $1-H(X)$를 $p_2$로 치환하면 다음과 같다

$ -(y_1 log(p_1) + y_2 log(p_2)) $

$ = - ( \displaystyle\sum_{i=1}^{2} y_i log(p_i)) = - ( \displaystyle\sum_{i=1}^{k} y_i log(p_i)) $

결과적으로 k=2인 소프트맥스 함수와 같다

## Softmax Regression 구현

In [62]:
# 훈련 데이터
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


### low-level 구현

$ H(X) = softmax(WX + B) $

$ cost(W) = - \frac{1}{n} \displaystyle\sum_{i=1}^{n} \displaystyle\sum_{j=1}^{k} y_j^{(i)} log(p_j^{(i)}) $

In [63]:
# y_train에서 원-핫 인코딩을 한 결과이여야 하고, 클래스 개수는 3개, y_train에 원-핫 인코딩한 결과는 8 × 3
y_one_hot = torch.zeros(8, 3)
y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)

# y_train에서 원-핫 인코딩을 한 결과인 y_one_hot의 크기는 8 × 3, W 행렬의 크기는 4 × 3
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [64]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) 

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.623000
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285073
Epoch 1000/1000 Cost: 0.248155


### high-level 구현

In [65]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros((1, 3), requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [66]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # cost 계산
    # F.cross_entropy() 내부에 softmax가 포함
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.622999
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285073
Epoch 1000/1000 Cost: 0.248155


### nn.Module 로 구현

In [67]:
# 모델을 선언 및 초기화. 4개의 특성을 가지고 3개의 클래스로 분류. input_dim=4, output_dim=3.
model = nn.Linear(4, 3)

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [68]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.849513
Epoch  100/1000 Cost: 0.689894
Epoch  200/1000 Cost: 0.609259
Epoch  300/1000 Cost: 0.551218
Epoch  400/1000 Cost: 0.500141
Epoch  500/1000 Cost: 0.451947
Epoch  600/1000 Cost: 0.405051
Epoch  700/1000 Cost: 0.358733
Epoch  800/1000 Cost: 0.312912
Epoch  900/1000 Cost: 0.269522
Epoch 1000/1000 Cost: 0.241922


### class 로 구현

In [69]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

In [70]:
# W 크기
model = SoftmaxClassifierModel(4, 3)

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [71]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print(f'Epoch {epoch:4d}/{nb_epochs} Cost: {cost.item():.6f}')

Epoch    0/1000 Cost: 1.845720
Epoch  100/1000 Cost: 0.647150
Epoch  200/1000 Cost: 0.568868
Epoch  300/1000 Cost: 0.515699
Epoch  400/1000 Cost: 0.471727
Epoch  500/1000 Cost: 0.432486
Epoch  600/1000 Cost: 0.395879
Epoch  700/1000 Cost: 0.360507
Epoch  800/1000 Cost: 0.325227
Epoch  900/1000 Cost: 0.289217
Epoch 1000/1000 Cost: 0.254086
